In [26]:
# Imports
from collections import defaultdict

In [27]:
# Day 1
with open("input_01.txt") as f:
    data = f.read().splitlines()

instructions = data[0].split(", ")
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
visited = {(0, 0)}
position = ((0, 1), (0, 0))
answer0102 = None

for instruction in instructions:
    turn = instruction[0]
    steps = int(instruction[1:])
    current_direction = position[0]
    
    new_direction = directions[(directions.index(current_direction) + (1 if turn == 'R' else -1)) % 4]
    for step in range(steps):
        position = (new_direction, (position[1][0] + new_direction[0], position[1][1] + new_direction[1]))
        if position[1] in visited and answer0102 is None:
            answer0102 = abs(position[1][0]) + abs(position[1][1])
        visited.add(position[1])

answer0101 = abs(position[1][0]) + abs(position[1][1])

In [28]:
# Day 2
with open("input_02.txt") as f:
    data = f.read().splitlines()

# (r, c)
small_keypad = {
    (0, 0): '1',
    (0, 1): '2',
    (0, 2): '3',
    (1, 0): '4',
    (1, 1): '5',
    (1, 2): '6',
    (2, 0): '7',
    (2, 1): '8',
    (2, 2): '9' 
}

big_keypad = {
    (0, 2): '1',
    (1, 1): '2',
    (1, 2): '3',
    (1, 3): '4',
    (2, 0): '5',
    (2, 1): '6',
    (2, 2): '7',
    (2, 3): '8',
    (2, 4): '9',
    (3, 1): 'A',
    (3, 2): 'B',
    (3, 3): 'C',
    (4, 2): 'D'
}

moves = {
    'U': (-1, 0),
    'D': (1, 0),
    'L': (0, -1),
    'R': (0, 1)
}

answer0201 = ""
answer0202 = ""
small_position = (1, 1)
big_position = (2, 0)

for instruction in data:
    for move in instruction:
        new_small = (small_position[0] + moves[move][0], small_position[1] + moves[move][1])
        new_big = (big_position[0] + moves[move][0], big_position[1] + moves[move][1])
        small_position = new_small if new_small in small_keypad.keys() else small_position
        big_position = new_big if new_big in big_keypad.keys() else big_position

    answer0201 = answer0201 + small_keypad[small_position]
    answer0202 = answer0202 + big_keypad[big_position]

In [29]:
# Day 3
with open("input_03.txt") as f:
    data = f.read().splitlines()

answer0301 = 0
answer0302 = 0

column_triangles = [[], [], []]

for line in data:
    A, B, C = [int(x) for x in line.split()]  
    if A + B > C and A + C > B and B + C > A:
        answer0301 += 1
    
    column_triangles[0].append(A)
    column_triangles[1].append(B)
    column_triangles[2].append(C)
    if len(column_triangles[0]) == 3:
        for i in range(3):
            A, B, C = column_triangles[i]
            if A + B > C and A + C > B and B + C > A:
                answer0302 += 1
        column_triangles = [[], [], []]

In [30]:
# Day 4
with open("input_04.txt") as f:
    data = f.read().splitlines()

answer0401 = 0
answer0402 = 0

for room in data:
    letter_count = defaultdict(int)
    checksum_check = ''
    
    encrypted_name, checksum = room.split("[")
    checksum = checksum[:-1]
    sector_id = ''

    for letter in encrypted_name:
        if letter >= 'a' and letter <= 'z':
            letter_count[letter] += 1
        elif letter != '-':
            sector_id += letter
    sector_id = int(sector_id)
    
    while len(checksum_check) < 5:
        top_values = sorted([x for x in letter_count.values()])
        candidates = [k for k, v in letter_count.items() if v == top_values[-1]]
        next_letter = sorted(candidates, reverse = True)[-1]
        checksum_check = checksum_check + next_letter
        letter_count[next_letter] = 0

    if checksum == checksum_check:
        answer0401 += sector_id

print(answer0401)

361724


In [31]:
add = 343
word = ''
text = 'qzmt-zixmtkozy-ivhz'
for letter in text:
    if letter == '-':
        pass
    else:
        word = word + chr(ord(letter) + add % 26 * 26)

print(word)

óüïöüëúïöíñüûëøêü


In [32]:
letter1 = 'a'
letter2 = 'b'
print(chr(ord(letter2) + 25 % 25))

b


In [33]:
# Day 5
with open("input_05.txt") as f:
    data = f.read().splitlines()

answer0501 = 0
answer0502 = 0

In [34]:
# Day 6
with open("input_06.txt") as f:
    data = f.read().splitlines()

answer0601 = 0
answer0602 = 0

In [35]:
# Day 7
with open("input_07.txt") as f:
    data = f.read().splitlines()

answer0701 = 0
answer0702 = 0

In [36]:
# Day 8
with open("input_08.txt") as f:
    data = f.read().splitlines()

answer0801 = 0
answer0802 = 0

In [37]:
# Day 9
with open("input_09.txt") as f:
    data = f.read().splitlines()

answer0901 = 0
answer0902 = 0

In [38]:
# Day 10
with open("input_10.txt") as f:
    data = f.read().splitlines()

data = """
value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2"""

answer1001 = 0
answer1002 = 0

In [39]:
# Day 11
with open("input_11.txt") as f:
    data = f.read().splitlines()

answer1101 = 0
answer1102 = 0

In [40]:
# Day 12
with open("input_12.txt") as f:
    data = f.read().splitlines()

answer1201 = 0
answer1202 = 0

In [41]:
# Day 13
with open("input_13.txt") as f:
    data = f.read().splitlines()

answer1301 = 0
answer1302 = 0

In [42]:
# Day 14
with open("input_14.txt") as f:
    data = f.read().splitlines()

answer1401 = 0
answer1402 = 0

In [43]:
# Day 15
with open("input_15.txt") as f:
    data = f.read().splitlines()

answer1501 = 0
answer1502 = 0

In [44]:
# Day 16
with open("input_16.txt") as f:
    data = f.read().splitlines()

answer1601 = 0
answer1602 = 0

In [45]:
# Day 17
with open("input_17.txt") as f:
    data = f.read().splitlines()

answer1701 = 0
answer1702 = 0

In [46]:
# Day 18
with open("input_18.txt") as f:
    data = f.read().splitlines()

answer1801 = 0
answer1802 = 0

In [47]:
# Day 19
with open("input_19.txt") as f:
    data = f.read().splitlines()

answer1901 = 0
answer1902 = 0

In [48]:
# Day 20
with open("input_20.txt") as f:
    data = f.read().splitlines()

answer2001 = 0
answer2002 = 0

In [49]:
# Day 21
with open("input_21.txt") as f:
    data = f.read().splitlines()

answer2101 = 0
answer2102 = 0

In [50]:
# Day 22
with open("input_22.txt") as f:
    data = f.read().splitlines()

answer2201 = 0
answer2202 = 0

In [51]:
# Day 23
with open("input_23.txt") as f:
    data = f.read().splitlines()

answer2301 = 0
answer2302 = 0

In [52]:
# Day 24
with open("input_24.txt") as f:
    data = f.read().splitlines()

answer2401 = 0
answer2402 = 0

In [53]:
# Day 25
with open("input_25.txt") as f:
    data = f.read().splitlines()

answer2501 = 0
answer2502 = 0

In [ ]:
# Answers
def pretty_answer(day, part):
    day_text = ("00" + str(day))[-2:]
    part_text = ("00" + str(part))[-2:]
    answer_text = str(globals()["answer" + day_text + part_text])
    output_text = "Day " + day_text + ", part " + part_text + ": " + answer_text
    print(output_text)

for d in range(1, 26):
    for p in range(1, 3):
        pretty_answer(d, p)
    print()

ANSWERS
Day 01, part 01: 243
Day 01, part 02: 142

Day 02, part 01: 74921
Day 02, part 02: A6B35

Day 03, part 01: 869
Day 03, part 02: 1544

Day 04, part 01: 361724
Day 04, part 02: 0

Day 05, part 01: 0
Day 05, part 02: 0

Day 06, part 01: 0
Day 06, part 02: 0

Day 07, part 01: 0
Day 07, part 02: 0

Day 08, part 01: 0
Day 08, part 02: 0

Day 09, part 01: 0
Day 09, part 02: 0

Day 10, part 01: 0
Day 10, part 02: 0

Day 11, part 01: 0
Day 11, part 02: 0

Day 12, part 01: 0
Day 12, part 02: 0

Day 13, part 01: 0
Day 13, part 02: 0

Day 14, part 01: 0
Day 14, part 02: 0

Day 15, part 01: 0
Day 15, part 02: 0

Day 16, part 01: 0
Day 16, part 02: 0

Day 17, part 01: 0
Day 17, part 02: 0

Day 18, part 01: 0
Day 18, part 02: 0

Day 19, part 01: 0
Day 19, part 02: 0

Day 20, part 01: 0
Day 20, part 02: 0

Day 21, part 01: 0
Day 21, part 02: 0

Day 22, part 01: 0
Day 22, part 02: 0

Day 23, part 01: 0
Day 23, part 02: 0

Day 24, part 01: 0
Day 24, part 02: 0

Day 25, part 01: 0
Day 25, part 02